In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [9]:
def loadParms(featureCount):
    parms = {}
    parms["featureCount"]  = featureCount
    parms["l1Size"]     = 6
    parms["l2Size"]     = 3
    parms["l3Size"]     = 2
    parms["l4Size"]     = parms["l2Size"]
    parms["l5Size"]     = parms["l1Size"]
    parms["lr"]         = .0002
    #parms["Lambda"]     = 10e-5          # Regularization parameter
    parms["batchSize"]  = 50
    parms["epochs"]     = 10
    parms["activation"] = tf.nn.tanh           # 'tanh' 'leakyReLU' 'ReLU' 'relu6' 'elu' 'crelu'
    parms["stdDev"]     = [0.2]
    return parms

In [10]:
def getData():
    folder = "/home/tbrownex/data/test/"
    file   = "autoEncoder.csv"
    df = pd.read_csv(folder+file,index_col=0)
    cols = ["v1","v2","v3","v4","v5","v6","v7","v8","v9","label"]
    df.columns = cols
    return df

In [11]:
def prepData(df):
    train, test = train_test_split(df, test_size=0.2)
    train = train[train['label'] == 0]     # Train on "normal" data only
    del train["label"]
    del test["label"]
    
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train)
    test  = scaler.transform(test)
    train, val = train_test_split(train, test_size=0.1)
    return train, val, test

In [15]:
class Model:
    def __init__(self, parms):
        self._cost = None
        self._train = None
        
        self.X  = tf.placeholder("float", shape=[None, featureCount], name="input")

        self.l1w = tf.Variable(tf.truncated_normal([parms["featureCount"], parms["l1Size"]], stddev=parms["stdDev"], dtype=tf.float32))
        self.l2w = tf.Variable(tf.truncated_normal([parms["l1Size"], parms["l2Size"]], stddev=parms["stdDev"], dtype=tf.float32))
        self.l3w = tf.Variable(tf.truncated_normal([parms["l2Size"], parms["l3Size"]], stddev=parms["stdDev"], dtype=tf.float32))
        self.l4w = tf.Variable(tf.truncated_normal([parms["l3Size"], parms["l4Size"]], stddev=parms["stdDev"], dtype=tf.float32))
        self.l5w = tf.Variable(tf.truncated_normal([parms["l4Size"], parms["l5Size"]], stddev=parms["stdDev"], dtype=tf.float32))
        self.l6w = tf.Variable(tf.truncated_normal([parms["l5Size"], featureCount], stddev=parms["stdDev"], dtype=tf.float32))

        self.l1b = tf.Variable(tf.zeros(parms["l1Size"]))
        self.l2b = tf.Variable(tf.zeros(parms["l2Size"]))
        self.l3b = tf.Variable(tf.zeros(parms["l3Size"]))
        self.l4b = tf.Variable(tf.zeros(parms["l4Size"]))
        self.l5b = tf.Variable(tf.zeros(parms["l5Size"]))
        self.l6b = tf.Variable(tf.zeros(featureCount))
        
        actf = parms["activation"]
        self.lr = parms["lr"]
        
        l1Out = actf(tf.matmul(self.X,self.l1w)+self.l1b)
        l2Out = actf(tf.matmul(l1Out,self.l2w)+self.l2b)
        l3Out = actf(tf.matmul(l2Out,self.l3w)+self.l3b)
        l4Out = actf(tf.matmul(l3Out,self.l4w)+self.l4b)
        l5Out = actf(tf.matmul(l4Out,self.l5w)+self.l5b)
        self.output = actf(tf.matmul(l5Out,self.l6w)+self.l6b)
    
    @property
    def cost(self):
        if self._cost is None:
            self._cost = tf.reduce_mean(tf.square(self.output - self.X))
        return self._cost

    @property
    def train(self):
        if self._train is None:
            optimizer = tf.train.AdamOptimizer(self.lr)
            self._train = optimizer.minimize(self.cost)
        return self._train

In [16]:
df = getData()
trainData, valData, testData = prepData(df)
featureCount = trainData.shape[1]

/home/tbrownex/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [17]:
parms = loadParms(featureCount)
m = Model(parms)
training = m.train
valCost  = m.cost

In [18]:
batchSize = 50
epochs    = 50
num_training_batches = int(trainData.shape[0] / batchSize)
print("{} training batches of size {}".format(num_training_batches, batchSize))

63 training batches of size 50


In [19]:
bestCost = 99.
earlyStopping = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        data = shuffle(trainData)
        for j in range(num_training_batches):
            x_mini = trainData[j*batchSize:j*batchSize+batchSize]
            _ = sess.run(training, feed_dict = {m.X: x_mini})
        if i%5 ==0:
            valLoss = sess.run(valCost, feed_dict={m.X:valData})
            if valLoss/bestCost > 0.003:
                bestCost = valLoss
                earlyStopping = 0
            else:
                earlyStopping += 1
        if earlyStopping > 3:    # no improvement for a while so stop
            break
    testLoss, recreation = sess.run([valCost, m.output], feed_dict={m.X:testData})
    print("{}{:.3f}".format("Reconstruction error (mse): ", testLoss))

Reconstruction error (mse): 0.027
